In [1]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib

import SatCallbacks
import Clone_Model
import rsc

In [5]:
#tf.compat.v1.disable_eager_execution()

In [2]:
checkpoint_path = "./cloning_eager_dis/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning_eager_dis/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [14]:
#      CLONE MODEL EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)

#K.clear_session() 

#tf.compat.v1.disable_eager_execution()

batch_size = None #doesnt work/conflict tensorboard_callback
train, test = rsc.get_titanic_dataset()



model = rsc.get_model(None)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


tensorboard_callback.set_model(model)

history = model.fit(train,
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[])


clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

sat_cb = Clone_Model.sat_results()


history = clone.fit(train,
                    batch_size=batch_size,
                    epochs=1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[sat_cb])
                

Train on 3 steps, validate on 1 steps
3/3 [==============================] - 0s 23ms/step - loss: 3.1866 - val_loss: 1.7910
RESET Weights!
RESET Weights!
RESET Weights!
RESET Weights!
RESET Weights!


ValueError: Error when checking input: expected model_input to have 2 dimensions, but got array with shape (1, 28, 28)

In [3]:
#      CLASSIFY MNIST EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)

K.clear_session() 
print(tf.executing_eagerly())

#problem source
#tf.compat.v1.disable_eager_execution()

batch_size = 20 

train, test = rsc.get_mnist()

model = rsc.get_model_unitlist(hidden_layers_spec=[128])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
    run_eagerly = False
    )


tensorboard_callback.set_model(model)

history = model.fit(train[0],
                    train[1],
                    batch_size=batch_size,
                    epochs=5, 
                    steps_per_epoch=None, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[])


True
Epoch 1/5
3000/3000 [==============================] - 9s 3ms/step - loss: 2.1774 - accuracy: 0.8586 - val_loss: 0.2160 - val_accuracy: 0.9000
Epoch 2/5
3000/3000 [==============================] - 7s 2ms/step - loss: 0.3746 - accuracy: 0.9093 - val_loss: 0.1072 - val_accuracy: 1.0000
Epoch 3/5
3000/3000 [==============================] - 7s 2ms/step - loss: 0.3141 - accuracy: 0.9233 - val_loss: 0.0371 - val_accuracy: 1.0000
Epoch 4/5
3000/3000 [==============================] - 11s 4ms/step - loss: 0.2804 - accuracy: 0.9323 - val_loss: 0.0280 - val_accuracy: 1.0000
Epoch 5/5
3000/3000 [==============================] - 11s 4ms/step - loss: 0.2653 - accuracy: 0.9349 - val_loss: 0.0337 - val_accuracy: 1.0000


In [7]:
#      CLONE MNIST MODEL EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)
importlib.reload(SatCallbacks)
#K.clear_session() 

#This mixed v1 v2 TF and can only cause problems.
#-> pass eager=False to model.fit() instead!!
#tf.compat.v1.disable_eager_execution()
print(tf.executing_eagerly())

#with tf.distribute.OneDeviceStrategy(device="/cpu:0").scope():

clone = Clone_Model.satify_model(model, {'run_eagerly' : False})

sat_cb = Clone_Model.sat_results()

new_cb = SatCallbacks.sat_logger()

history = clone.fit(train[0],
                    train[1],
                    batch_size=10,
                    epochs=1,
                    steps_per_epoch=5,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[new_cb])


True
<class 'tensorflow.python.keras.layers.core.Flatten'>
input_shape: [None, 128]
layer_width: (128,)
input_shape: [None, 10]
layer_width: (10,)
1/5 [=====>........................] - ETA: 0s - loss: 3.9791 - accuracy: 1.0000cross-replica-ctxt: False
tf.dist.strat:  <tensorflow.python.distribute.distribute_lib._DefaultDistributionStrategy object at 0x7f2bb4e1e0d0>

Comparing batch_k and batch_tf:

Shape batch_k: [ 10 128]
Shape batch_tf: [ 10 128]
Tensors batch_k and batch_tf are equal! 
 


Comparing batch_k_0 and batch_tf_0:

Shape batch_k_0: [128]
Shape batch_tf_0: [128]
Tensors batch_k_0 and batch_tf_0 are equal! 
 


Comparing batch_k_1 and batch_tf_1:

Shape batch_k_1: [128]
Shape batch_tf_1: [128]
Tensors batch_k_1 and batch_tf_1 are equal! 
 


Comparing batch_k_2 and batch_tf_2:

Shape batch_k_2: [128]
Shape batch_tf_2: [128]
Tensors batch_k_2 and batch_tf_2 are equal! 
 


Comparing batch_k_3 and batch_tf_3:

Shape batch_k_3: [128]
Shape batch_tf_3: [128]
Tensors batch_k_3 

In [19]:
print(history.history.keys())
print(history.__dict__)

dict_keys(['loss', 'accuracy', 'o_s_sl_metric', 'val_loss', 'val_accuracy', 'val_o_s_sl_metric'])
{'validation_data': None, 'model': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fabba92cf90>, '_chief_worker_only': None, 'history': {'loss': [5.974883079528809], 'accuracy': [0.9833333492279053], 'o_s_sl_metric': [18.28329982904939], 'val_loss': [4.267309665679932], 'val_accuracy': [1.0], 'val_o_s_sl_metric': [48.28329982904939]}, 'params': {'verbose': 1, 'epochs': 1, 'steps': 5}, 'epoch': [0]}


In [ ]:
""" Warum macht tf probleme und zeigt auf auskommentierten code?
        #Default to k=1 if first EVal explains delta of variance alone 
        #TODO change condition so graph mode works without tf.function
        #if not 
        #    #cond = tf.constant([True], dtype=tf.bool)
            #print("cond with default catch: {}".format(cond))   
        """    
        
    
    #k = max(len(tf.where(cond)), 1 ) #TODO use min(1,tf.where()) instead of checking above
        
        #print("tf.where(): {}".format(tf.where(cond)))
        #print("K: in ll_fun {}".format(k))
       